In [1]:
from data_model import *
from decimal import Decimal
from pprint import pprint
from time import time
from copy import deepcopy
import pandas as pd # should not be used
pd.set_option('display.max_columns', 50)

import re, os, pickle
import itertools
import random, string
import collections
from utils import *

In [2]:
import itertools
import collections

In [9]:
PATH = "/Users/sasha/Work/Projects/matching_experiment/data/OCBC2_PK/"
# PATH = "/Users/sasha/Work/Projects/matching_experiment/data/OCBC_TEST_PK/"
FILES = os.listdir(PATH)
pprint(FILES)

['Day 1 ToolKitaki (Mask)-given.pkl',
 'Day 4 ToolKitaki (Mask)-given.pkl',
 'Day 3 ToolKitaki (Mask)-given.pkl',
 'Batch3-Remasked-Day 3.pkl',
 'Batch3-Remasked-Day 2.pkl',
 'Day 2 ToolKitaki (Mask)-given.pkl',
 'Batch3-Remasked-Day 0.pkl',
 'Batch3-Remasked-Day 1.pkl']


In [10]:
file_dict = {str(i):FILES[i] for i in range(len(FILES))}

In [12]:
arr_all = []
for i in range(len(FILES)):
    # if FILES[i] == 'User Set 1.pkl':
    if not (FILES[i] in IGNORE):
        with open(PATH + FILES[i], 'rb') as f:
            arr = pickle.load(f)
            #print(FILES[i])
        print(len(arr))
        arr_ = []
        for x in arr:
            # adding day ID to set_id and match_id
            arr_.append(Transaction(set_id = x.set_id + "|" + str(i),
                                    amount = x.amount, 
                                    type_id= x.type_id,
                                    match_id= x.match_id + "|" + str(i),
                                    item_id = x.item_id + "|" + str(i),
                                    ref_key = x.ref_key,
                                    sl_type=x.sl_type,
                                   tt_match_id=x.tt_match_id))
        arr_all += arr_
print("Total number of trans:", len(arr_all))
LN0 = len(arr_all)

17531
34218
34207
39918
16093
17159
0
22789
Total number of trans: 181915


In [13]:
# data = pd.DataFrame(arr_all, columns=['Set ID', 'Amount', 'Type', 'Match ID', 'Item ID', 'REF_KEY', 'SL_type', 'TT_MATCH_KEY']) 
# PATH2 = "/Users/sasha/Work/Projects/matching_experiment/data/OCBC2_EX/"
# data.to_excel(PATH2 + 'SET_1_BLIND.xlsx', index=False, header = True)

In [14]:
# removing zero-amount transaction
arr_all = [x for x in arr_all if x.amount != 0]

In [15]:
# arr_all[:10]

In [16]:
# pattern = r"IMXMX\d{3} \d{2}-[A-Z]+-\d{2}.*-\d{8}-[A-Z\s_]+-[A-Z0-9]+"
# pattern = r"^IMYMU\d{3} .*\d{8}\;[A-Z0-9_]\;[A-Z0-9_]"
# pattern = r"^IMYMU\d{3} .*\d{8};[^(BOND)][A-Z0-9_]+"
pattern =  r"^ISLRD002 \d{2}-[A-Z]+-\d{2} \d{3} RETTIL (AANKING|AANNKXPUNIN)"
#pattern = r"^IMYMU\d{3} .*;(A287_FX_COMM_TIMEOPT|A311_FX_SWP);[A-Z0-1]+"
out = []
for x in arr_all:
    match = re.search(pattern, x.ref_key)
    if match:
        xs = match.group()
        print(xs)
        # out.append(xs.split(";")[1])
    #if match:
    #    key = match.group().split("-")[-2]
    #    out.append(key)

ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANNKXPUNIN
ISLRD002 05-JUL-11 104 RETTIL AANN

In [141]:
collections.OrderedDict(collections.Counter(out).most_common())

OrderedDict([('A311_FX_SWP', 882),
             ('A287_FX_COMM_TIMEOPT', 278),
             ('A285_FX_COMM_FWD', 166),
             ('A286_FX_SPOT', 79),
             ('A286_FX_SPOT_SARF', 18),
             ('A285_FX_FWD_WAAD', 15),
             ('A286_FX_COMM_SPOT', 15),
             ('A285_FX_COMM_NDFDCI', 12),
             ('A256_INTERAANK_LEND', 10),
             ('A316_IRS', 9),
             ('A240_INTERAANK_BORR', 9),
             ('A240_IBK_BOR_QARD', 8),
             ('A314_CROSS_CCY_SWAP', 7),
             ('A492_STMMD', 5),
             ('A312_FX_INV_SWP', 5),
             ('A311_FX_SWP_WAAD', 4),
             ('A285_FX_NDFDCI_SARF', 2),
             ('F321_CDS_CLI', 1),
             ('A285_FX_FWD', 1)])

In [93]:
collections.OrderedDict(collections.Counter(out).most_common())

OrderedDict([('FXD ', 5252),
             ('EQUIT', 1024),
             ('SCF ', 1009),
             ('BOND ', 869),
             ('IXRXS ', 466),
             ('IRS ', 364),
             ('EQS ', 206),
             ('CS ', 72),
             ('CF ', 58),
             ('OPT ', 47),
             ('REPOX ', 12),
             ('REPO ', 4),
             ('CRDI ', 3),
             ('OSWP ', 2)])

In [49]:
# list of systems
first_word = [x.ref_key.split()[0] for x in arr_all if x.sl_type == "L" and not re.search(r"^\d{3}",x.ref_key)]
xs = collections.Counter(first_word)
ys = collections.OrderedDict(xs.most_common())
ys

OrderedDict([('ITDRM001', 3761),
             ('IMXMX004', 2346),
             ('IMXMX002', 2267),
             ('IMXMX003', 2187),
             ('IMXMX001', 2091),
             ('ICNCP001', 1565),
             ('IMYEB001', 1209),
             ('ISLRD002', 1039),
             ('IIBAS001', 957),
             ('IMYPM001', 842),
             ('SPREADSHEET', 582),
             ('IMYMU004', 461),
             ('IMYMU001', 442),
             ('IMYMU002', 419),
             ('IMYMU003', 406),
             ('ITDSR001', 286),
             ('ISLLN001', 281),
             ('ITDRM002', 241),
             ('ITDCF001', 174),
             ('ISLRD001', 157),
             ('IMYTF001', 156),
             ('IMYRD002', 126),
             ('IMYCC001', 114),
             ('IUTUT001', 110),
             ('ISCMY001', 79),
             ('IMYRM001', 68),
             ('ITDBG001', 60),
             ('ICNTF001', 44),
             ('IMXPY001', 43),
             ('ITDFX002', 41),
             ('IMYFX001', 34),
    

In [7]:
arr_mtch2, arr_all2 = single_dr_cr(arr_all)

Number of matched transactions: 7983
Correct: 7983 (100.0%)
Incorrect: 0 (0.0%)


In [8]:
arr_mtch3, arr_all3 = unique_pair(arr_all2)

Number of matched transactions: 6576
Correct: 6506 (98.94%)
Incorrect: 0 (0.0%)


In [9]:
arr_mtch4, arr_all4 = single_dr_cr(arr_all3)

Number of matched transactions: 3087
Correct: 3087 (100.0%)
Incorrect: 0 (0.0%)


In [10]:
arr_mtch5, arr_all5 = eliminate_threes(arr_all4, cut_off = 50)

Number of matched transactions: 582
Correct: 576 (98.97%)
Incorrect: 6 (1.03%)


In [11]:
arr_mtch6, arr_all6 = single_dr_cr(arr_all5)

Number of matched transactions: 61
Correct: 61 (100.0%)
Incorrect: 0 (0.0%)


In [12]:
arr_mtch7, arr_all7 = unique_pair(arr_all6)

Number of matched transactions: 0


In [13]:
arr_all8 = match_sl_by_unq_key(arr_all6)

Conducting repartitioning
Total number pre-matched: 3156
Correctly pre-matched: 3073 (97.37%)
262
522


In [14]:
arr_mtch9, arr_all9 = unique_pair(arr_all8)

Number of matched transactions: 100
Correct: 80 (80.0%)
Incorrect: 18 (18.0%)


In [15]:
arr_mtch10, arr_all10 = single_dr_cr(arr_all9)

Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, only Transactions are supported
Currently, 

In [16]:
arr_mtch11, arr_all11 = eliminate_threes(arr_all10, cut_off = 50)

Number of matched transactions: 6
Correct: 6 (100.0%)
Incorrect: 0 (0.0%)


In [17]:
arr_mtch12, arr_all12 = unique_pair(arr_all11)

Number of matched transactions: 0


In [18]:
arr_mtch13, arr_all13 = single_dr_cr(arr_all12)

Number of matched transactions: 0


In [9]:
reg_expr = r"ISLIG001 \d{2}-[A-Z]{3}-\d{2} 104 IBG A \d{8} * "
xs = match_ledger(arr_all2, reg_expr, lambda x: x.split()[5])
xs
# arr_inw = create_uk(arr_all13, reg_expr, lambda x: x.split()[5])
# print("Number of transactions covered:", len(arr_inw))
# consist_un = check_consistency(arr_inw, exclude_unique = True)
# consist_nn = check_consistency(arr_inw, exclude_unique = False)
# print("Consistency without singles: ", consist_un)
# l1 = consist_un[0]
# print("Consistency with singles: ", consist_nn)

All component are disjoint


[PreMatch(item_ids={'400496444_0', '400496443_0'}, unq_key={'34949960'}),
 PreMatch(item_ids={'400501034_0', '400501035_0', '400501051_0', '400501041_0', '400501049_0', '400501053_0'}, unq_key={'34949960'}),
 PreMatch(item_ids={'398847092_0', '398846890_0', '398847440_0', '398846991_0', '398847444_0', '398847114_0', '398847184_0', '398846985_0', '398847441_0', '398846874_0', '398846981_0'}, unq_key={'34932760'}),
 PreMatch(item_ids={'400502146_0', '400501985_0', '400502066_0', '400501698_0', '400502145_0', '400501755_0', '400501793_0', '400501908_0', '400502144_0', '400501736_0', '400501898_0'}, unq_key={'34949960'}),
 PreMatch(item_ids={'400502306_0', '400502316_0', '400502293_0', '400502314_0', '400502313_0', '400502315_0', '400502300_0', '400502323_0', '400502312_0', '400502294_0', '400502303_0', '400502299_0', '400502307_0', '400502301_0', '400502324_0', '400502288_0', '400502289_0', '400502308_0', '400502296_0', '400502297_0', '400502290_0', '400502310_0', '400502319_0', '40050231

In [26]:
arr_inw = create_uk(arr_all13, r"A \d{8} INW CLRG-\d{3}-", lambda x: x.split()[1])
print("Number of transactions covered:", len(arr_inw))
consist_un = check_consistency(arr_inw, exclude_unique = True)
consist_nn = check_consistency(arr_inw, exclude_unique = False)
print("Consistency without singles: ", consist_un)
l2 = consist_un[0]
print("Consistency with singles: ", consist_nn)

Number of transactions covered: 3326
Number of complete matches: 0
Number of complete matches: 0
Consistency without singles:  (3326, 0, 3326)
Consistency with singles:  (3326, 0, 3326)


In [27]:
reg_expr = r"A \d{8} \d{16}BILL PAYMENT VIA A33XS TERMINAL"
arr_inw = create_uk(arr_all13, reg_expr, lambda x: x.split()[1])
print("Number of transactions covered:", len(arr_inw))
consist_un = check_consistency(arr_inw, exclude_unique = True)
consist_nn = check_consistency(arr_inw, exclude_unique = False)
print("Consistency without singles: ", consist_un)
l3 = consist_un[0]
print("Consistency with singles: ", consist_nn)

Number of transactions covered: 2528
Number of complete matches: 0
Number of complete matches: 0
Consistency without singles:  (2528, 0, 2528)
Consistency with singles:  (2528, 0, 2528)


In [28]:
#PET453708149AC
reg_expr = r"PET\d{9}"
arr_inw = create_uk(arr_all13, reg_expr)
print("Number of transactions covered:", len(arr_inw))
consist_un = check_consistency(arr_inw, exclude_unique = True)
consist_nn = check_consistency(arr_inw, exclude_unique = False)
print("Consistency without singles: ", consist_un)
l4 = consist_un[0]
print("Consistency with singles: ", consist_nn)

Number of transactions covered: 57
Number of complete matches: 4
Number of complete matches: 0
Consistency without singles:  (4, 3, 7)
Consistency with singles:  (54, 3, 57)


In [29]:
print("Coverage is:", 100 - round((len(arr_all13) - l1 - l2 - l3 - l4) / LN0 * 100, 2))

Coverage is: 70.24
